In [1]:
#'''
#Todo: Add more lines in the relation C:\Users\win7\Documents\Fonte_data\sourcePort_to_shipperId.csv
#then test the program.
#Need to write a summary for smelted quantity of each material.
#
#'''


In [15]:
# Import
import pandas as pd
import numpy as np
import os
from datetime import date

In [16]:
# Read starting inventory file


### Open transaction data
The transaction file is obtained in Horne web page/ Applications/ Rapports Horne/ MAIT/ SIGI/ Transactions de materiel.  
Take the transactions from yesterday to today.  
Transactions sorted by time because some ports are both a destination and source, and we want to track their composition.  
To self: possibly use all transactions starting from a certain date. Accounts for transaction changes.

In [18]:
# Read transaction file.
dT = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\Transactions de matériel 1er dec.csv')
# Drop useless columns
dT = dT.loc[:,['TAS_ID', 'REVERSE_TAS_ID1', 'Textbox58', 'SHIPPER_CODE','ACTION_TIME']]
# Rename columns
dT.columns = ['sourcePort', 'destPort', 'DMT', 'shipperCode','time']
# Sort by time
dT = dT.sort_values(by=['time'])

dT.head(15)

,sourcePort,destPort,DMT,shipperCode,time
85,600,3008,"1,11",600,531
87,600,3008,"10,57",600,531
90,600,730,"0,09",600,531
86,600,3008,"7,33",600,531
100,600,798,"0,59",600,634
97,600,798,"18,63",600,634
125,841,798,"12,07",841,936
88,600,3008,"18,71",600,1105
89,600,3008,"18,56",600,1205
92,600,730,"0,07",600,1205


In [21]:
# Convert comma to decimal
dT.replace(',','.', regex=True, inplace=True)
# Convert to float
dT.loc[:,'DMT'] = pd.to_numeric(dT.loc[:,'DMT'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
85,600,3008,1.11,600,531
87,600,3008,10.57,600,531
90,600,730,0.09,600,531
86,600,3008,7.33,600,531
100,600,798,0.59,600,634


In [25]:
# Keep rows if destination is to "Eglise"
# Todo eglisePorts read from csv
eglisePorts = [564,565,900,903,906,913,915,916,917,918,919,920,921,922,923,924]

dT = dT[dT['destPort'].isin(eglisePorts)]
dT.head()

,sourcePort,destPort,DMT,shipperCode,time
64,322,564,398.77,,62350
67,373,564,71.82,,62357
68,376,564,21.93,,62404
76,500,564,487.51,,62412
80,564,916,787.16,,62419


We want a unique identifier for each material, therefore we wish to convert the port numbers to an alphabetical shipper code.
shipperCode column currently contains both string shipper codes ('EAGL', 'STAL', etc.) and port numbers (812, 376, etc.). 

In [39]:
# shipperCode column contains port numbers and shipper codes
# Replace empty strings with NaN
dT['shipperCode'] = dT['shipperCode'].replace(r'^\s*$', np.nan, regex=True)
dT.head()

,sourcePort,destPort,DMT,shipperCode,time,shipperID
64,322,564,398.77,NaN,62350,MTA
67,373,564,71.82,NaN,62357,Electronique Rx
68,376,564,21.93,NaN,62404,LSN_et_INC4
76,500,564,487.51,NaN,62412,TMO
80,564,916,787.16,NaN,62419,Melange fil


Now we load a dictionnary to assign a shipper code to each line of dT that has a port number value in the column 'shipperCode'.  
However, some port numbers are actually a mix of materials (ex: 304 is a mix of green concentrates, 385 is a mix of concentrates with nickel).  
If a port is composed of a single type of material, we treat it as a single material even if it is a mix of different materials (ex: all 304 is MTA despite having MTA, XKM, BCM, COPA, FLI).  
If not, we consider its composition (ex: 564 can be a mix of concentrates, pyrite and electronics).


In [40]:
# Load the dictionnary to replace numerical port numbers with shipper codes. shipper_code = f(source_port)
# This dictionnary must be up to date, as port numbers and materials may change.

shipID = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\sourcePort_to_shipperId.csv')
shipID.head()


,ShipperID,SourcePort
0,BCM,NaN
1,Pyrite_don_rouyn_gallen,498.0
2,NMP,520.0
3,NMP,521.0
4,XKM,NaN


In [41]:
# Remove row if SourcePort value is missing
shipID = shipID.loc[shipID['SourcePort'].notnull(),:]
shipID.shape

(73, 2)

In [42]:
# Create dictionary
shipID_dict = shipID.set_index('SourcePort')['ShipperID'].to_dict()
# This also works
# shipID_dict = dict(zip(shipID['SourcePort'], shipID['ShipperID']))

In [43]:
# Map shipperID to transactions
dT['shipperID'] = dT['sourcePort'].map(shipID_dict)
# Drop null values of shipperID
#dT = dT[dT['shipperID'].notnull()]
dT.head()

,sourcePort,destPort,DMT,shipperCode,time,shipperID
64,322,564,398.77,NaN,62350,MTA
67,373,564,71.82,NaN,62357,Electronique Rx
68,376,564,21.93,NaN,62404,LSN_et_INC4
76,500,564,487.51,NaN,62412,TMO
80,564,916,787.16,NaN,62419,Melange fil


In [45]:
dT[dT['shipperID'].isnull()]

,sourcePort,destPort,DMT,shipperCode,time,shipperID
36,20001,903,37.25,DUM,75948,NaN
176,20002,906,86.83,EAGL,91450,NaN
227,802,923,3.00,802,100137,NaN
223,778,923,0.42,778,100825,NaN
229,802,923,3.28,802,101105,NaN
111,802,923,4.20,802,102951,NaN
103,778,923,5.33,778,103751,NaN
178,20002,906,86.77,EAGL,104047,NaN
177,20002,906,88.27,EAGL,104065,NaN
110,802,923,4.28,802,105204,NaN


In [47]:
# if shipperID is null, and if shipperCode is notnull, copy shipperCode to shipperID
# Copies over the shipperCode given by the transaction data.
dT['shipperID'] = np.where(dT['shipperCode'].notnull() & dT['shipperID'].isnull(),dT['shipperCode'], dT['shipperID'])
dT.head()

,sourcePort,destPort,DMT,shipperCode,time,shipperID
64,322,564,398.77,NaN,62350,MTA
67,373,564,71.82,NaN,62357,Electronique Rx
68,376,564,21.93,NaN,62404,LSN_et_INC4
76,500,564,487.51,NaN,62412,TMO
80,564,916,787.16,NaN,62419,Melange fil


In [53]:
# Check for remaining transactions (Transactions still without a shipperID).
dT[dT['shipperID'].isnull()]


,sourcePort,destPort,DMT,shipperCode,time,shipperID


In [54]:
print(dT.to_string())

     sourcePort  destPort     DMT shipperCode    time        shipperID
64          322       564  398.77         NaN   62350              MTA
67          373       564   71.82         NaN   62357  Electronique Rx
68          376       564   21.93         NaN   62404      LSN_et_INC4
76          500       564  487.51         NaN   62412              TMO
80          564       916  787.16         NaN   62419      Melange fil
82          564       917  198.35         NaN   62427      Melange fil
66          354       917   55.73         NaN   62434             CAPE
83          566       918  108.17         NaN   62441              CUZ
84          589       918  103.25         NaN   62447             STAL
63          312       919    5.64         NaN   62453         Internes
71          438       919  286.57         NaN   62459              CLH
72          487       919   73.01         NaN   62516             BOLA
79          534       919   32.56         NaN   62521             BOLA
70    

In [ ]:
# Check if shipperCode value is a letter string (not numeric)
# Todo: Display rows containing alpha shipper codes.
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()
#non_numeric_rows = ~dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()

In [ ]:
#temp2 = dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()
#temp2
# find a way to select rows that are notnull and not numeric...

In [ ]:
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode'].str.isnumeric()

In [ ]:
#dT.loc[dT.loc[:,'shipperCode'].notnull(),'shipperCode']

In [ ]:
#dT[non_numeric_rows]

In [ ]:
# Find rows containing non-numeric and not-null
#dT.loc[non_numeric_rows,:]

In [ ]:
# ShipperCode column contains port numbers and shipper codes. 
# If it is a letter string, copy this to the shipperID column
# is not numeric how to find
#dT[dT.loc[:,'shipperCode'].notnull().isnumeric()

Read inventory.   
This is the starting inventory at Eglise, on the Monday of the week. Data must be entered manually. Names on the first column must exactly match the shipperID nomenclature, otherwise transactions will be omitted.

In [51]:
# Read prior inventory
inv = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\inv2.csv', index_col=0)
inv = inv.fillna(0)

inv

,564,560,900,903,906,913,915,916,917,918,919,920,921,922,923,924,925
XKM,367.0,0.0,264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.0,0.0,604.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,0.0,0.0,954.0,0.0,0.0,0.0,0.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BCM,366.0,0.0,178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DUM,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NVA,110.0,0.0,0.0,44.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LSN_et_INC4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TIZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The order of transaction matters, from oldest to newest.

In [52]:
# Add transactions to inventory
# This is the naive method, find a more elegant way later.
for k in range(dT.shape[0]):
    for j in inv.columns:
        #If the sourcePort is 564, and the destPort matches with inv column, consider the composition of 564
        if dT.iloc[k,0]==564 and str(dT.iloc[k,1])==j:
            # Remaining mass fraction in 564
            m = 1 - dT.loc[dT.index[k], 'DMT'] / inv.loc[:,'564'].sum()
            # Add mass to destPort
            inv.loc[:,j] += (1-m)*inv.loc[:,'564']
            print('Transferred {:.2f} tons from 564 to {}.'.format((1-m)*inv.loc[:,'564'].sum(),j))
            # Remove mass from 564
            inv.loc[:,'564'] = m * inv.loc[:,'564']
            
        else:
            for i in inv.index:
                # If shipperID and port number matches
                if dT.iloc[k,-1] == i and str(dT.iloc[k,1])==j:
                    inv.loc[i,j] += float(dT.iloc[k,2])
                    print('Added '+str(dT.iloc[k,2])+' tons of '+str(i)+' to port '+str(j)+'.')

            
                
inv.head()


Added 398.77 tons of MTA to port 564.
Added 21.93 tons of LSN_et_INC4 to port 564.
Added 487.51 tons of TMO to port 564.
Transferred 787.16 tons from 564 to 916.
Transferred 198.35 tons from 564 to 917.
Added 108.17 tons of CUZ to port 918.
Added 103.25 tons of STAL to port 918.
Added 286.57 tons of CLH to port 919.
Added 73.01 tons of BOLA to port 919.
Added 32.56 tons of BOLA to port 919.
Added 151.0 tons of NMP to port 920.
Added 26.47 tons of WOR to port 921.
Transferred 19.15 tons from 564 to 916.
Added 32.13 tons of MTA to port 564.
Added 993.23 tons of TMO to port 564.
Transferred 677.05 tons from 564 to 916.
Transferred 447.47 tons from 564 to 917.
Added 36.99 tons of CLH to port 918.
Added 125.35 tons of CUZ to port 918.
Added 108.2 tons of STAL to port 918.
Added 119.61 tons of CLH to port 919.
Added 36.97 tons of BOLA to port 919.
Added 27.7 tons of BOLA to port 919.
Added 23.58 tons of WOR to port 920.
Transferred 15.22 tons from 564 to 916.
Added 350.0 tons of LSN_et_INC4 

,564,560,900,903,906,913,915,916,917,918,919,920,921,922,923,924,925
XKM,98.158337,0.0,264.0,0.0,0.0,0.0,0.0,195.808096,73.033567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FLI,0.000000,0.0,604.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MTA,122.392873,0.0,954.0,0.0,0.0,0.0,0.0,288.715385,85.791742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pyrite_don_rouyn_gallen,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BCM,97.890876,0.0,178.0,0.0,0.0,0.0,0.0,195.274558,72.834565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Check if any of the values are negative.
(inv.values<0).any()

In [ ]:
inv

In [ ]:
# Read smelted transactions. This data is from the daily reactor report (rapport du reacteur)
smelt = pd.read_csv(r'C:\Users\win7\Documents\Fonte_data\smelt_2021-12-15.csv')
smelt.head()

In [ ]:
# Remove mass from inv according the smelt data
# Save all removed mass in a separate DataFrame
smelted = pd.Series(data=0, index=inv.index)
for row in range(len(smelt.SourcePort)):
    for inv_port in inv.columns:
        if str(smelt.loc[row,'SourcePort']) == inv_port:
            if smelt.loc[row,'DMT']>0:
                # Skip the inventory update for the port if it is empty.
                # Because it is impossible to know which material to assign negative mass in the port.
                if ~(inv.loc[:,inv_port]!=0).any(axis=0):
                    print(('Skipped removing inventory from port {} because it is empty.'
                          ' Cannot smelt from an empty port').format(inv_port))
                    continue
                # Calculate remaining fraction in the Eglise Port
                m = 1 - smelt.loc[row,'DMT'] / inv.loc[:,inv_port].sum()
                # Calculate removed mass by shipperID
                smelted += (1-m)*inv.loc[:,inv_port]
                # Update inventory
                inv.loc[:,inv_port] = inv.loc[:,inv_port] * m
                print('Removed {} dmt from port {}'.format(smelt.loc[row,'DMT'], inv_port))
                if m<0:
                    print('Caution, negative inventory in port {}: {} dmt.'.format(inv_port, inv.loc[:,inv_port].sum()))
                
                
inv.sum(axis=0)
        

Save smelted mass by shipperID

In [ ]:
smelted.to_csv('smelted'+date.today().strftime('%d-%m-%Y')+'.csv')

In [ ]:

smelted.sum()

In [ ]:
smelt['DMT'].sum()

In [ ]:
'smelted'+date.today().strftime('%d-%m-%Y')+'.csv'